In [1]:
from utils import *
import pandas as pd

In [2]:
df_mme = pd.read_csv("data/M.csv")

In [3]:
df = df_mme.pipe(clean_data).query("Fecha >= '2022-01-01'")
df_despacho = df.pipe(return_despacho_movements)
df_hogar = df.pipe(return_hogar_movements)

In [4]:
df_mme.query("Estado != 'R'")

,Identificador,Fecha,Estado,Tipo,Cuenta,Beneficiario,Categoría,Importe,Divisa,Número,Notas


In [6]:
pd.concat([
    (df_despacho
     .pipe(pivot_by_category, ["Tipo", "Categoria", "Subcategoria", "Beneficiario"])
     .query("Tipo == 'Gastos' and Total > 0")
    ),
    (df_hogar
     .pipe(pivot_by_category, ["Tipo", "Categoria", "Subcategoria", "Beneficiario"])
     .query("Tipo == 'Gastos' and Total > 0")
    )
])

,Tipo,Categoria,Subcategoria,Beneficiario,Diciembre 2023,Noviembre 2023,Octubre 2023,Septiembre 2023,Agosto 2023,Julio 2023,...,Julio 2022,Junio 2022,Mayo 2022,Abril 2022,Marzo 2022,Febrero 2022,Enero 2022,Media,Total,Enero 2024


In [8]:
(df_despacho
     .pipe(pivot_by_category, ["Tipo", "Categoria", "Subcategoria", "Beneficiario"])
     .query("Tipo == 'Gastos' and Total > 0")
)

,Tipo,Categoria,Subcategoria,Beneficiario,Diciembre 2023,Noviembre 2023,Octubre 2023,Septiembre 2023,Agosto 2023,Julio 2023,...,Agosto 2022,Julio 2022,Junio 2022,Mayo 2022,Abril 2022,Marzo 2022,Febrero 2022,Enero 2022,Media,Total


In [9]:
df.sample(5)

,Fecha,Tipo,Categoria,Subcategoria,Beneficiario,Transaccion,Importe
978,2023-04-25,Ingresos,Ingresos,Nómina,SANTANDER TECNOLOGIA,Abono,2765.87
456,2022-08-31,Gastos,Extras,Viajes,TORREVIEJA,Cargo,-10.4
847,2023-03-21,Gastos,Gastos Fijos,Clases Inglés,SHIRA COHEN,Cargo,-32.0
2322,2022-11-18,Gastos,Supervivencia,Farmacia,FARMACIA ALCORCON,Cargo,-6.8
744,2022-12-31,Gastos,Extras,Viajes,ITALIA,Cargo,-4.25


In [10]:
(df_hogar
    .query("Fecha.dt.year == 2023")
    .pipe(pivot_by_category_totals, ["Tipo"])
       .pipe(style_locale_es)
 )

,Tipo,Diciembre 2023,Noviembre 2023,Octubre 2023,Septiembre 2023,Agosto 2023,Julio 2023,Junio 2023,Mayo 2023,Abril 2023,Marzo 2023,Febrero 2023,Enero 2023,Media,Total
0,Gastos,"-6.819,05","-5.695,97","-5.186,94","-6.488,95","-2.599,50","-6.898,82","-7.654,93","-8.133,37","-13.608,77","-4.665,30","-5.744,66","-5.170,56","-6.555,57","-78.666,82"
1,Ingresos,"7.990,44","5.184,09","6.174,62","3.912,53","3.432,44","11.288,66","7.642,24","9.800,26","5.024,67","5.023,47","11.008,56","2.811,73","6.607,81","79.293,71"
2,Total,"1.171,39","-511,88","987,68","-2.576,42","832,94","4.389,84","-12,69","1.666,89","-8.584,10","358,17","5.263,90","-2.358,83","52,24","626,89"


In [11]:
(df_despacho
    .query("Fecha.dt.year == 2023")
    .pipe(pivot_by_category_totals, ["Tipo"])
    .pipe(style_locale_es)
 )

,Tipo,Diciembre 2023,Noviembre 2023,Octubre 2023,Septiembre 2023,Agosto 2023,Julio 2023,Junio 2023,Mayo 2023,Abril 2023,Marzo 2023,Febrero 2023,Enero 2023,Media,Total
0,Gastos,"-442,67","-403,52","-833,30","-224,84","-219,22","-2.010,02","-370,45","-411,15","-578,48","-353,14","-547,56","-1.102,12","-624,71","-7.496,47"
1,Ingresos,"2.583,83","2.773,02","1.012,68","1.330,96","848,00","7.931,74","1.639,38","4.200,80","2.794,74","2.586,80","2.262,19","1.086,68","2.587,57","31.050,82"
2,Total,"2.141,16","2.369,50","179,38","1.106,12","628,78","5.921,72","1.268,93","3.789,65","2.216,26","2.233,66","1.714,63","-15,44","1.962,86","23.554,35"
